In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [2]:
import pandas as pd

### bus_list

In [32]:
webdriver_path = r"C:\Users\peipeipig\OneDrive\桌面\Python\AI-project\chromedriver.exe"

In [51]:
driver = webdriver.Chrome(executable_path=webdriver_path)

C:\Users\peipeipig\AppData\Local\Temp\ipykernel_14884\4005181481.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=webdriver_path)


In [4]:
driver.get("https://ebus.gov.taipei/Route/StopsOfRoute?routeid=0100026520")

In [5]:
route_obj = driver.execute_script("return objArr")

In [11]:

pd.DataFrame(route_obj['BackDirStops'])[["PublicId", "Name", "Latitude", "Longitude"]].assign(
    bus = "265"
)

,PublicId,Name,Latitude,Longitude,bus
0,1005100140,成功中學(林森),25.042924,121.522689,265
1,1005100560,立法院,25.043292,121.519461,265
2,1004100760,臺北車站(忠孝),25.046477,121.517183,265
3,1084300041,中華路北站,25.044584,121.508765,265
4,1084600340,捷運西門站,25.041649,121.507887,265
5,1085000340,小南門,25.038008,121.506812,265
6,1084900160,桂林昆明街口,25.038100,121.504300,265
7,1084800160,桂林路,25.038472,121.500815,265
8,1085500540,龍山寺(西園),25.036110,121.499270,265
9,1085500040,大理服飾,25.034146,121.498448,265


In [52]:
driver.get("https://ebus.gov.taipei/ebus?ct=all")

In [53]:
elements = driver.find_elements(By.CSS_SELECTOR, '.busline a')

In [56]:
elements[0].get_attribute('innerHTML')

'0東 '

In [58]:
list_of_already_scraped = set(pd.read_csv('bus_station_name.csv')['Bus_name'].unique())


elements = [e for e in elements if 
    e.get_attribute("innerHTML").strip() not in list_of_already_scraped]

In [59]:
len(elements)

748

In [8]:
bus_list = []

try:
    # Find elements with class="busline" and <a tag
    # elements = driver.find_elements(By.CSS_SELECTOR, '.busline a')

    # Extract and add each element to the list
    for element in elements:
        name = element.get_attribute("innerHTML")
        bus_id = element.get_attribute("href")
        bus_list.append(dict(name=name, bus_id=bus_id))

finally:
    ...
    # Close the browser window
    # driver.quit()

# Print the list of elements
print(bus_list)

[{'name': '0東 ', 'bus_id': "javascript:go('0100000A00')"}, {'name': '0南 ', 'bus_id': "javascript:go('0100000B00')"}, {'name': '1 ', 'bus_id': "javascript:go('0100000100')"}, {'name': '2 ', 'bus_id': "javascript:go('0100000200')"}, {'name': '5 ', 'bus_id': "javascript:go('0100000500')"}, {'name': '9 ', 'bus_id': "javascript:go('0100000900')"}, {'name': '12 ', 'bus_id': "javascript:go('0100001200')"}, {'name': '14 ', 'bus_id': "javascript:go('0100001400')"}, {'name': '18 ', 'bus_id': "javascript:go('0100001800')"}, {'name': '20 ', 'bus_id': "javascript:go('0100002000')"}, {'name': '21 ', 'bus_id': "javascript:go('0100002100')"}, {'name': '22 ', 'bus_id': "javascript:go('0100002200')"}, {'name': '26 ', 'bus_id': "javascript:go('0100002600')"}, {'name': '28 ', 'bus_id': "javascript:go('0100002800')"}, {'name': '32 ', 'bus_id': "javascript:go('0100003200')"}, {'name': '33 ', 'bus_id': "javascript:go('0100003300')"}, {'name': '37 ', 'bus_id': "javascript:go('0100003700')"}, {'name': '38 ', '

In [9]:
import re

with open("bus_list.txt", 'a', encoding='utf-8-sig') as f:
    for bus in bus_list:
        name = bus['name'].strip()
        bus_id_js = bus['bus_id']
        match = re.search(r"\'(.+?)\'", bus_id_js)
        bus_id = match.group(1)
        f.write(name + ", " + bus_id + '\n')

### Bus_station

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [4]:
bus_list = pd.read_csv('bus_list.txt', sep = ', ')

bus_list_unique = bus_list.drop_duplicates(subset = 'bus_id')

bus_list_unique

C:\Users\peipeipig\AppData\Local\Temp\ipykernel_14884\1836090160.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  bus_list = pd.read_csv('bus_list.txt', sep = ', ')


,name,bus_id
0,0東,0100000A00
1,0南,0100000B00
2,1,0100000100
3,2,0100000200
4,5,0100000500
...,...,...
535,貓空左線(指南宮),0162002100
536,貓空左線(動物園),0162002200
537,臺北觀光巴士紅線,0162003400
538,臺北觀光巴士藍線,0162003500


In [6]:
webdriver_path = r"C:\Users\peipeipig\OneDrive\桌面\Python\AI-project\chromedriver.exe"

driver = webdriver.Chrome(executable_path=webdriver_path)

C:\Users\peipeipig\AppData\Local\Temp\ipykernel_14884\402819173.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=webdriver_path)


In [7]:
bus_station = pd.DataFrame(columns=['PublicId', 'Name', 'Latitude', 'Longitude', 'bus_id'])

for bus in bus_list_unique['bus_id']:
    driver.get(f"https://ebus.gov.taipei/Route/StopsOfRoute?routeid={bus}")
    # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "some_element_id")))

    route_obj = driver.execute_script("return objArr")

    # Check if route_obj is not None
    if route_obj is not None:
        # Try to get 'BackDirStops'; if not found, try 'GoDirStops'
        back_dir_stops = route_obj.get('BackDirStops') or route_obj.get('GoDirStops')

        # Check if the required columns exist in the DataFrame
        if back_dir_stops is not None:
            df = pd.DataFrame(back_dir_stops)[["PublicId", "Name", "Latitude", "Longitude"]].assign(
                bus_id=bus)
            bus_station = pd.concat([bus_station, df])
            print(f"bus_id: {bus} processed")
        else:
            print(f"bus_id: {bus} doesn't contain the required columns")
    else:
        print(f"bus_id: {bus} has no data")

bus_station

bus_id: 0100000A00 processed
bus_id: 0100000B00 processed
bus_id: 0100000100 processed
bus_id: 0100000200 processed
bus_id: 0100000500 processed
bus_id: 0100000900 processed
bus_id: 0100001200 processed
bus_id: 0100001400 processed
bus_id: 0100001800 processed
bus_id: 0100002000 processed
bus_id: 0100002100 processed
bus_id: 0100002200 processed
bus_id: 0100002600 processed
bus_id: 0100002800 processed
bus_id: 0100003200 processed
bus_id: 0100003300 processed
bus_id: 0100003700 processed
bus_id: 0100003800 processed
bus_id: 0100003820 processed
bus_id: 0100003900 processed
bus_id: 0100003940 processed
bus_id: 0100004100 processed
bus_id: 0100004200 processed
bus_id: 0100004220 processed
bus_id: 0100004600 processed
bus_id: 0100004900 processed
bus_id: 0100005200 processed
bus_id: 0100005300 processed
bus_id: 0100006300 processed
bus_id: 0100006600 processed
bus_id: 0100006800 processed
bus_id: 0100007200 processed
bus_id: 0100008800 processed
bus_id: 0100008820 processed
bus_id: 010001

,PublicId,Name,Latitude,Longitude,bus_id
0,1004700340,臺北車站(公園),25.043929,121.516775,0100000A00
1,1004600040,捷運台大醫院站,25.041935,121.516148,0100000A00
2,1004800920,中正紀念堂,25.037153,121.520771,0100000A00
3,1004800820,信義杭州路口,25.035700,121.523836,0100000A00
4,1064100120,捷運東門站,25.034134,121.526994,0100000A00
...,...,...,...,...,...
26,1166000340,國泰新村(興隆),24.982543,121.562265,0162000400
27,1165800360,國泰新村(木新),24.982053,121.561850,0162000400
28,1166000060,力行國小,24.981660,121.560289,0162000400
29,1166000160,木新市場,24.981153,121.558272,0162000400


In [10]:
bus_station_name = pd.merge(bus_station, bus_list_unique, on = 'bus_id', how = 'left')

bus_station_name = bus_station_name.rename(columns = {'name': 'Bus_name'})

bus_station_name = bus_station_name[['PublicId', 'Name', 'Latitude', 'Longitude', 'Bus_name']]

bus_station_name

,PublicId,Name,Latitude,Longitude,Bus_name
0,1004700340,臺北車站(公園),25.043929,121.516775,0東
1,1004600040,捷運台大醫院站,25.041935,121.516148,0東
2,1004800920,中正紀念堂,25.037153,121.520771,0東
3,1004800820,信義杭州路口,25.035700,121.523836,0東
4,1064100120,捷運東門站,25.034134,121.526994,0東
...,...,...,...,...,...
13620,1166000340,國泰新村(興隆),24.982543,121.562265,景美-榮總(快)
13621,1165800360,國泰新村(木新),24.982053,121.561850,景美-榮總(快)
13622,1166000060,力行國小,24.981660,121.560289,景美-榮總(快)
13623,1166000160,木新市場,24.981153,121.558272,景美-榮總(快)


In [74]:
import pandas as pd

# bus_station_name.to_csv('bus_station_name.csv', index = False)
bus_station_name = pd.read_csv("bus_station_name.csv")

In [75]:
def examine_bus_stop(df:pd.DataFrame,  station_name:str, bus_name:str=None):
    if bus_name is None:
        mask = df['Name'].str.contains(station_name, regex=False)
    else:
        mask = (df['Bus_name'] == bus_name) & (df['Name'].str.contains(station_name, regex=False))
    return df[mask]

In [76]:
# Create a pivot table to count occurrences
pivot_table = bus_station_name.pivot_table(index=['PublicId', 'Name', 'Latitude', 'Longitude'], values='Bus_name', aggfunc='count')

# Reset index to make the result a DataFrame
bus_agg = pivot_table.reset_index().rename(columns = {'Bus_name': 'Bus_number'})

bus_agg

,PublicId,Name,Latitude,Longitude,Bus_number
0,1000100040,捷運中正紀念堂站(中山),25.036430,121.516730,1
1,1000500040,重慶南路一段,25.045607,121.513187,2
2,1000500041,重慶南路一段,25.044691,121.513165,2
3,1000500042,重慶南路一段,25.044432,121.513114,2
4,1000900060,衡陽路,25.042270,121.510210,5
...,...,...,...,...,...
3694,2517206110,崁頂三路,25.195630,121.430072,1
3695,2517206260,輕軌淡水行政中心站,25.189169,121.443384,1
3696,2517300400,馬偕醫院,25.141655,121.459785,4
3697,2517300500,捷運竹圍站,25.136591,121.460120,4


In [4]:
len(bus_agg['PublicId'].unique())

3699

In [5]:
ddd = examine_bus_stop(bus_station_name, '捷運中正紀念堂站')
ddd

,PublicId,Name,Latitude,Longitude,Bus_name
309,1009300100,捷運中正紀念堂站(羅斯福),25.032297,121.518848,18
3059,1006600820,捷運中正紀念堂站(勞保局),25.033718,121.517306,251
3163,1006600820,捷運中正紀念堂站(勞保局),25.033718,121.517306,252
4828,1007400340,捷運中正紀念堂站(羅斯福),25.030999,121.519394,295
4874,1007400340,捷運中正紀念堂站(羅斯福),25.030999,121.519394,295副
7164,1006600820,捷運中正紀念堂站(勞保局),25.033718,121.517306,644
7410,1000100040,捷運中正紀念堂站(中山),25.036430,121.516730,648
7411,1006600820,捷運中正紀念堂站(勞保局),25.033718,121.517306,648
7567,1006600820,捷運中正紀念堂站(勞保局),25.033718,121.517306,660
10356,1007400340,捷運中正紀念堂站(羅斯福),25.030999,121.519394,和平幹線


In [77]:
import pyproj
import numpy as np

In [78]:
lat_lon_crs = 'EPSG:4326'  # WGS84, standard lat/lon CRS
target_crs  = 'EPSG:3826'  # EPSG code for your target CRS

transformer = pyproj.Transformer.from_crs(lat_lon_crs, target_crs, always_xy=True)
bus_agg['x'], bus_agg['y'] = transformer.transform( bus_agg['Longitude'], bus_agg['Latitude'])

In [79]:
bus_agg

,PublicId,Name,Latitude,Longitude,Bus_number,x,y
0,1000100040,捷運中正紀念堂站(中山),25.036430,121.516730,1,302143.790748,2.769912e+06
1,1000500040,重慶南路一段,25.045607,121.513187,2,301782.392692,2.770927e+06
2,1000500041,重慶南路一段,25.044691,121.513165,2,301780.557479,2.770826e+06
3,1000500042,重慶南路一段,25.044432,121.513114,2,301775.530123,2.770797e+06
4,1000900060,衡陽路,25.042270,121.510210,5,301483.401044,2.770557e+06
...,...,...,...,...,...,...,...
3694,2517206110,崁頂三路,25.195630,121.430072,1,293342.737269,2.787515e+06
3695,2517206260,輕軌淡水行政中心站,25.189169,121.443384,1,294686.697500,2.786804e+06
3696,2517300400,馬偕醫院,25.141655,121.459785,4,296357.683439,2.781547e+06
3697,2517300500,捷運竹圍站,25.136591,121.460120,4,296393.333479,2.780986e+06


In [164]:
def get_agg_around_point(df:pd.DataFrame, x:np.ndarray, y:np.ndarray, dis:float):
    stacked = np.stack([x, y], axis=-1)
    bus_np = df[['x', 'y']].to_numpy()
    close = np.linalg.norm(                                 # B * Y
                    bus_np[:, np.newaxis, :] - stacked
            ,axis= -1) < dis
    num_buses = df['Bus_number'].to_numpy()[:, np.newaxis]      # B * 1

    return np.sum(num_buses * close, axis=0)

In [155]:
fake_close = np.array([
    [1,1,0,0,0],
    [0,1,0,0,1],
    [1,1,1,1,1]]
).T

num_of_bus = np.array([1,2,3,4,5])#[:, np.newaxis]

In [143]:
fake_close

array([[1, 0, 1],
       [1, 1, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 1]])

In [156]:
np.sum(fake_close * num_of_bus, axis=0)

ValueError: operands could not be broadcast together with shapes (5,3) (5,) 

In [160]:
fake_close.T

array([[1, 1, 0, 0, 0],
       [0, 1, 0, 0, 1],
       [1, 1, 1, 1, 1]])

In [157]:
num_of_bus

array([1, 2, 3, 4, 5])

In [163]:
np.dot(fake_close.T, num_of_bus)

array([ 3,  7, 15])

In [165]:
get_agg_around_point(bus_agg, 296358.561333, 2781322, 20000)

array([12644], dtype=int64)

In [80]:
import requests as rq

yb_info = rq.get("https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json").json()
yb_df = pd.DataFrame(yb_info)
# yb_info

In [81]:
yb_df['x'], yb_df['y'] = transformer.transform( yb_df['lng'], yb_df['lat'])

In [166]:
get_agg_around_point(bus_agg, x=yb_df['x'], y= yb_df['y'], dis=500)

array([80, 81, 78, ..., 51, 66, 70], dtype=int64)

In [132]:
%%timeit
yb_df['bus_arround'] = yb_df.apply(
    lambda row: get_agg_around_point(bus_agg, row['x'], row['y'], 500), axis=1)

2.55 s ± 67.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
demo_around = yb_df[['lat', 'lng', 'sna', 'bus_arround']]
demo_around

,lat,lng,sna,bus_arround
0,25.02605,121.54360,YouBike2.0_捷運科技大樓站,80
1,25.02565,121.54357,YouBike2.0_復興南路二段273號前,81
2,25.02429,121.54124,YouBike2.0_國北教大實小東側門,78
3,25.02351,121.54282,YouBike2.0_和平公園東側,95
4,25.02153,121.54299,YouBike2.0_辛亥復興路口西北側,84
...,...,...,...,...
1320,25.01690,121.54031,YouBike2.0_臺大總圖書館西南側,39
1321,25.01995,121.54347,YouBike2.0_臺大黑森林西側,56
1322,25.01791,121.54242,YouBike2.0_臺大獸醫館南側,51
1323,25.02112,121.53591,YouBike2.0_臺大新體育館東南側,66


### Numpy broadcast

In [1]:
import numpy as np

In [38]:
bus_coor = np.array([
    [121, 30],
    [120, 29],
    [122, 31]
])

In [68]:
a = np.ones((3,2))

In [73]:
a[:, np.newaxis, :]

array([[[1., 1.]],

       [[1., 1.]],

       [[1., 1.]]])

In [42]:
yb_coord = np.array([120, 30])

In [46]:
np.linalg.norm(bus_coor - yb_coord, axis=1)

array([1.        , 1.        , 2.23606798])

In [62]:
def get_yb_close_to_bus(x,y):
    yb_coord = np.stack([x,y], axis=-1)
    return np.linalg.norm(bus_coor[:, np.newaxis, :] - yb_coord, axis=-1)

In [65]:
x = np.array([121,120,121,122])
y= np.array([30,29,29,31])

get_yb_close_to_bus(x, y)

array([[0.        , 1.41421356, 1.        , 1.41421356],
       [1.41421356, 0.        , 1.        , 2.82842712],
       [1.41421356, 2.82842712, 2.23606798, 0.        ]])

In [37]:
print(a, end='\n===\n')
print(b)

[[[[1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]]]


 [[[1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]]]


 [[[1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]]]]
===
[[2. 2. 2. 2. 2.]]


In [36]:
a + b

array([[[[3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3.]]],


       [[[3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3.]]],


       [[[3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3.]]]])